In [3]:
from flask import Flask, request, jsonify
import requests
import firebase_admin
from firebase_admin import credentials
from google.cloud.firestore_v1.base_query import FieldFilter
from firebase_admin import firestore
import numpy as np  # Assuming you need to process the image with numpy

app = Flask(__name__)

# Replace this with the path to your Firebase service account key JSON file
cred = credentials.Certificate("C:/Users/aishi/OneDrive/Desktop/FR/snappy.json")

if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Replace this with the ML model endpoint URL
ml_model_endpoint = 'http://face_recognition'

@app.route('/process_image', methods=['POST'])
def process_image():
    try:
        # Assuming 'document_id' is provided in the request JSON
        document_id = request.json.get('document_id')

        # Retrieve the Firestore document by document_id
        doc_ref = db.collection('processed_images').document(document_id)
        doc = doc_ref.get()

        if doc.exists:
            # Extract the image URL from the Firestore document
            image_url = doc.to_dict().get('image', None)

            if image_url:
                # Make a request to the ML model endpoint with the image URL
                ml_response = requests.post(ml_model_endpoint, json={'image_url': image_url})

                # Process the ML model response (replace this with your actual processing code)
                response_json = ml_response.json()
                processed_data = response_json.get('result', None)

                # Store the processed data in Firestore
                new_doc_ref = db.collection('processed_images').add({
                    'processed_data': processed_data
                })

                return jsonify({'status': 'success', 'document_id': new_doc_ref.id})
            else:
                return jsonify({'status': 'error', 'message': 'Image URL not found in Firestore document'})
        else:
            return jsonify({'status': 'error', 'message': f'Document with id {document_id} not found'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'


 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1